# Test Audio Sender

This file is used for sending audio data to the device. It measures the accuracy based on individual audio files.

**Note that the software on the MCU must match this!**

In [9]:
AUDIO_FOLDER_PATH = "./audio"

In [6]:
import numpy as np
import pandas as pd
import gc
import os

In [24]:
audio_files = []
audio_file_classes = []

In [28]:
# Look each file in the audio folder
audio_class_folders = os.listdir(AUDIO_FOLDER_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(AUDIO_FOLDER_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the audio_files list
        audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        audio_file_classes.append(audio_class_folder)
print("audio_class_files length: ", len(audio_files[0:10]))
print("audio_file_classes length: ", len(audio_file_classes[0:10]))

Processing class folder:  ./audio/truck
Processing class folder:  ./audio/car
audio_class_files length:  8
audio_file_classes length:  8


In [29]:
# Loop each audio file.
for i in range(len(audio_class_files)):
    label = audio_file_classes[i]
    # Read the audio file
    audio_file = audio_files[i]
    # Check file format. Must be Wav.
    if audio_file.endswith(".wav"):
        # Load the audio file
        audio_data = np.load(audio_file)
        # Get the audio data shape
        audio_data_shape = audio_data.shape
        # Print the audio data shape
        print("Audio data shape: ", audio_data_shape)
        # Print the audio class
        print("Audio class: ", label)
        # Free memory
        del audio_data
        gc.collect()
    else:
        print("File format not supported: ", audio_file)
        continue
    

File format not supported:  ./audio/truck/tester.txt


In [4]:
df = pd.read_csv("test.csv", header=None)

In [5]:
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float32
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string
del df
gc.collect()

0

In [6]:
dataset = dataset.reshape(dataset.shape[0], 40, 16, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (290, 40, 16, 1)
Labels shape: (290,)
Input shape: (40, 16, 1)


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.95, random_state=42, stratify=labels_set)
del dataset, labels_set
gc.collect()

0

In [8]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))

2025-03-26 13:31:04.487152: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 13:31:04.552442: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 13:31:04.598885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742992264.646795   14992 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742992264.662852   14992 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742992264.754430   14992 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [9]:
def indexOfMax(arr):
    return np.argmax(arr)

In [16]:
import serial
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=115200,       # Set baud rate to 115200
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1              # Set timeout for reading
)

Configuring serial port...


In [17]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 115200 baud.


In [12]:
def calc_accuracy(y_true, y_pred):
    success = 0
    failure = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == int(indexOfMax(y_true[i])):
            success += 1
        else:
            failure += 1
    res = round(success / (success + failure), 4)
    return res

In [ ]:
import time
import struct

x = x_test
y = y_test[0:len(x)]
print(x.shape)

results = []
total = len(x)

# Send data
for i in range(total):
    data = x[i].flatten()
    for j in range(len(data)):
        data_to_send = struct.pack('f', data[j])
        ser.write(data_to_send)
    time.sleep(0.1)
    # Read response
    response = ser.readline().decode('utf-8').strip()
    results.append(response)
    print(f"Guess: {response} - Class: {indexOfMax(y[i])} Accuracy: {calc_accuracy(y, results)} ({i}/{total})")


Guess: 0 - Class: 0 Accuracy: 1.0 (0/276)
Guess: 3 - Class: 0 Accuracy: 0.5 (1/276)
Guess: 1 - Class: 0 Accuracy: 0.3333 (2/276)
Guess: 1 - Class: 0 Accuracy: 0.25 (3/276)
Guess: 3 - Class: 0 Accuracy: 0.2 (4/276)
Guess: 1 - Class: 0 Accuracy: 0.1667 (5/276)
Guess: 2 - Class: 0 Accuracy: 0.1429 (6/276)
Guess: 3 - Class: 0 Accuracy: 0.125 (7/276)
Guess: 3 - Class: 0 Accuracy: 0.1111 (8/276)
Guess: 1 - Class: 0 Accuracy: 0.1 (9/276)
Guess: 1 - Class: 0 Accuracy: 0.0909 (10/276)
Guess: 1 - Class: 0 Accuracy: 0.0833 (11/276)
Guess: 1 - Class: 0 Accuracy: 0.0769 (12/276)
Guess: 1 - Class: 0 Accuracy: 0.0714 (13/276)
Guess: 1 - Class: 0 Accuracy: 0.0667 (14/276)
Guess: 1 - Class: 0 Accuracy: 0.0625 (15/276)
Guess: 1 - Class: 0 Accuracy: 0.0588 (16/276)
Guess: 3 - Class: 0 Accuracy: 0.0556 (17/276)
Guess: 1 - Class: 0 Accuracy: 0.0526 (18/276)
Guess: 1 - Class: 0 Accuracy: 0.05 (19/276)
Guess: 2 - Class: 0 Accuracy: 0.0476 (20/276)
Guess: 2 - Class: 0 Accuracy: 0.0455 (21/276)
Guess: 1 - Clas

In [15]:
# Close the serial port
ser.close()